In [76]:
'''
Jiayou Qin 03/18/2024
SIFT based analyzer
'''

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
 
video_path = "../gpt/Scene_BikePath.mp4" #change this file name as needed
cap = cv.VideoCapture(video_path)


visual_intensity = 1
visualize_points = False
visualize_cursor = True

for i in range(15): #skip frames, must be larger than 1
    success, frame = cap.read()
# frame = cv.rotate(frame, cv.ROTATE_90_CLOCKWISE)
sift = cv.SIFT_create()
frame = cv.resize(frame,(800,600))
y,x = frame.shape[:2]
center = np.array([x/2,y/2], dtype=float)
cursor = center.copy()

last_frame = frame
kp, des = sift.detectAndCompute(frame,None)
cv.namedWindow('image',cv.WINDOW_NORMAL)
cv.imshow('image', frame)
cv.waitKey(0) #press key to begin

last_m = None
while True:
    for i in range(5):
        success, frame = cap.read()
    if not success:
        break
    angles = np.array([],dtype=float)
    delta = np.array([],dtype=float)
    pos = np.array([],dtype=float)
    frame = cv.resize(frame,(800,600))
#     frame = cv.rotate(frame, cv.ROTATE_90_CLOCKWISE)
    out_frame = frame.copy()
    if not success:
        break
        
    kp_,des_ = kp,des
    kp, des = sift.detectAndCompute(frame,None)
    bf = cv.BFMatcher()
    matches = bf.knnMatch(des,des_,k=2)
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            p1 = np.array(kp[m.queryIdx].pt, dtype=float)
            p2 = np.array(kp_[m.trainIdx].pt, dtype=float)
            delta = np.insert(delta, 0, p2-p1) #delta vector, indicating pixel change
            pos = np.insert(pos,0,p2)
            cv.circle(out_frame, p2.astype(int),2, (0,255,255) )
            cv.arrowedLine(out_frame, p2.astype(int),p1.astype(int), (0,0,255),1)
            
            
    pos = pos.reshape(-1,2)        
    delta = delta.reshape(-1,2)

    vainishing_point = np.array([400,300])
    
    v = vainishing_point - pos #vector from position to vainishing point
    magnitudes = np.sqrt((v ** 2).sum(-1))[..., np.newaxis]
    v/= magnitudes #normalized vector from pixel position to vainishing point
    magnitudes = np.sqrt((delta ** 2).sum(-1))[..., np.newaxis]
    delta /= magnitudes
    
    regression_value = np.sum(delta-v)  #regress this to minimal value
    cv.putText(out_frame, f'{regression_value}', (50,50), cv.FONT_HERSHEY_SIMPLEX , 1, (255,255,255), 2, cv.LINE_AA) 
    
    cv.imshow('image', out_frame)
    k = cv.waitKey(1)
    if k ==27:
        break
    
    last_frame = frame
    
#     break;
cap.release()
cv.destroyAllWindows()


In [57]:
regression_value

-1185.8293496488577